# 🎓 Agent Vocal IA - Setup Colab

Notebook d'installation et de vérification pour l'Agent Vocal IA éducatif 100% local.

**Fonctionnalités :**
- 🎤 ASR (Faster-Whisper + Silero VAD)
- 🔍 RAG (FAISS + SentenceTransformers)
- 🧠 LLM local (llama-cpp-python)
- 🔊 TTS (Piper-TTS)
- 💡 Système de hints progressifs

**Prérequis :** GPU T4 ou A100 recommandé

## 📋 Étape 1 : Vérification de l'environnement

Vérifions d'abord que nous avons bien accès à un GPU et les informations système.

In [ ]:
# Vérification GPU et CUDA
import subprocess
import sys
import platform

print("=" * 60)
print("🖥️  INFORMATIONS SYSTÈME")
print("=" * 60)
print(f"Python version: {sys.version}")
print(f"Plateforme: {platform.platform()}")
print(f"Architecture: {platform.machine()}")
print()

# Vérifier NVIDIA GPU
print("=" * 60)
print("🎮  VÉRIFICATION GPU")
print("=" * 60)
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print(result.stdout)
except Exception as e:
    print(f"⚠️  Aucun GPU NVIDIA détecté ou nvidia-smi non disponible: {e}")
    print("Note: L'exécution sera possible sur CPU mais beaucoup plus lente.")

print()
print("=" * 60)
print("📦  VÉRIFICATION PYTORCH")
print("=" * 60)
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"Nombre de GPUs: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"    Mémoire totale: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
except ImportError:
    print("PyTorch n'est pas encore installé. Il sera installé à l'étape suivante.")
print("=" * 60)

## 📥 Étape 2 : Clonage du dépôt (si nécessaire)

Si vous n'avez pas encore cloné le dépôt, décommentez et exécutez la cellule suivante.

In [ ]:
# Décommentez les lignes suivantes si vous devez cloner le dépôt
# !git clone https://github.com/Romainmlt123/agent_vocal_IA.git
# %cd agent_vocal_IA

# Si vous êtes déjà dans le dossier, vérifiez la structure
import os
print("📁 Dossier actuel:", os.getcwd())
print("\n📂 Structure du projet:")
for root, dirs, files in os.walk('.', topdown=True):
    # Limiter la profondeur pour la lisibilité
    level = root.replace('.', '').count(os.sep)
    if level < 3:
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            if not file.startswith('.'):
                print(f'{subindent}{file}')

## 📦 Étape 3 : Installation des dépendances

Installation de toutes les bibliothèques requises. **Cela peut prendre 5-10 minutes.**

In [ ]:
# Installation des dépendances depuis requirements.txt
print("🔄 Installation des packages...")
print("=" * 60)

# Installer les dépendances principales
!pip install -q -r requirements.txt

# Pour Colab, installer faiss-gpu au lieu de faiss-cpu pour de meilleures performances
print("\n📊 Installation de FAISS-GPU pour meilleures performances...")
!pip install -q faiss-gpu

# Vérifier llama-cpp-python avec support CUDA
print("\n🦙 Installation de llama-cpp-python avec support CUDA...")
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -q llama-cpp-python --upgrade --force-reinstall --no-cache-dir

print("\n✅ Installation terminée!")

## ✅ Étape 4 : Vérification des imports

Testons que tous les modules principaux peuvent être importés correctement.

In [ ]:
# Test des imports critiques
import sys

def test_import(module_name, display_name=None):
    """Test l'import d'un module et affiche le résultat."""
    if display_name is None:
        display_name = module_name
    try:
        module = __import__(module_name.split('.')[0])
        version = getattr(module, '__version__', 'version inconnue')
        print(f"✅ {display_name:30} - {version}")
        return True
    except ImportError as e:
        print(f"❌ {display_name:30} - ERREUR: {e}")
        return False

print("=" * 60)
print("📚 VÉRIFICATION DES BIBLIOTHÈQUES")
print("=" * 60)

modules_to_test = [
    ('torch', 'PyTorch'),
    ('torchaudio', 'TorchAudio'),
    ('faster_whisper', 'Faster-Whisper (ASR)'),
    ('silero_vad', 'Silero VAD'),
    ('sentence_transformers', 'SentenceTransformers'),
    ('faiss', 'FAISS'),
    ('pypdf', 'PyPDF'),
    ('langchain', 'LangChain'),
    ('llama_cpp', 'llama-cpp-python'),
    ('gradio', 'Gradio'),
    ('yaml', 'PyYAML'),
    ('numpy', 'NumPy'),
    ('soundfile', 'SoundFile'),
    ('sounddevice', 'SoundDevice'),
]

results = []
for module, name in modules_to_test:
    results.append(test_import(module, name))

print("=" * 60)
success_count = sum(results)
total_count = len(results)
print(f"\n📊 Résultat: {success_count}/{total_count} modules importés avec succès")

if success_count == total_count:
    print("🎉 Tous les modules sont prêts!")
else:
    print("⚠️  Certains modules ont échoué. Vérifiez les erreurs ci-dessus.")

## 📥 Étape 5 : Téléchargement des modèles

Téléchargement des modèles LLM et voix TTS. **Attention: peut prendre 10-15 minutes selon la connexion.**

In [ ]:
import os
import urllib.request
from pathlib import Path

def download_file(url, destination, description="Fichier"):
    """Télécharge un fichier avec barre de progression."""
    print(f"📥 Téléchargement de {description}...")
    print(f"   URL: {url}")
    print(f"   Destination: {destination}")
    
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    
    if os.path.exists(destination):
        print(f"   ✅ Fichier déjà présent, téléchargement ignoré.")
        return True
    
    try:
        def progress_hook(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(100, downloaded * 100 / total_size) if total_size > 0 else 0
            bar_length = 40
            filled = int(bar_length * percent / 100)
            bar = '█' * filled + '░' * (bar_length - filled)
            print(f'\r   [{bar}] {percent:.1f}%', end='', flush=True)
        
        urllib.request.urlretrieve(url, destination, progress_hook)
        print(f"\n   ✅ Téléchargement réussi!")
        return True
    except Exception as e:
        print(f"\n   ❌ Erreur: {e}")
        return False

print("=" * 60)
print("📦 TÉLÉCHARGEMENT DES MODÈLES")
print("=" * 60)

# Créer les dossiers nécessaires
Path("models/llm").mkdir(parents=True, exist_ok=True)
Path("models/voices").mkdir(parents=True, exist_ok=True)

print("\n🦙 Modèle LLM (Phi-3 Mini 4K Instruct - Q4_K_M)")
print("-" * 60)
llm_url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf"
llm_dest = "models/llm/phi-3-mini-4k-instruct.Q4_K_M.gguf"
download_file(llm_url, llm_dest, "Phi-3 Mini 4K (2.4 GB)")

print("\n\n🔊 Modèle TTS (Piper - Voix française Siwis)")
print("-" * 60)
tts_model_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx"
tts_config_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx.json"
tts_model_dest = "models/voices/fr_FR-siwis-medium.onnx"
tts_config_dest = "models/voices/fr_FR-siwis-medium.onnx.json"

download_file(tts_model_url, tts_model_dest, "Voix TTS française (60 MB)")
download_file(tts_config_url, tts_config_dest, "Config TTS")

print("\n" + "=" * 60)
print("🎉 Téléchargement des modèles terminé!")
print("=" * 60)

## 🧪 Étape 6 : Test rapide des modules

Testons rapidement chaque composant principal du système.

In [ ]:
# Test rapide de chaque composant
print("=" * 60)
print("🧪 TESTS DES COMPOSANTS")
print("=" * 60)

# Test 1: Configuration
print("\n1️⃣  Test de chargement de la configuration...")
try:
    import yaml
    with open('config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print(f"✅ Configuration chargée: {len(config)} sections")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 2: Faster-Whisper (ASR)
print("\n2️⃣  Test Faster-Whisper (ASR)...")
try:
    from faster_whisper import WhisperModel
    # Ne pas charger le modèle complet, juste vérifier l'import
    print("✅ Faster-Whisper disponible")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 3: SentenceTransformers (Embeddings)
print("\n3️⃣  Test SentenceTransformers (Embeddings)...")
try:
    from sentence_transformers import SentenceTransformer
    # Test d'embedding simple
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embedding = model.encode("Test de phrase")
    print(f"✅ Embeddings générés: dimension {len(embedding)}")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 4: FAISS
print("\n4️⃣  Test FAISS (Index vectoriel)...")
try:
    import faiss
    import numpy as np
    # Créer un petit index de test
    dimension = 384
    index = faiss.IndexFlatL2(dimension)
    vectors = np.random.random((10, dimension)).astype('float32')
    index.add(vectors)
    print(f"✅ FAISS opérationnel: {index.ntotal} vecteurs indexés")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 5: llama-cpp-python
print("\n5️⃣  Test llama-cpp-python (LLM)...")
try:
    from llama_cpp import Llama
    print("✅ llama-cpp-python disponible")
    # Vérifier si CUDA est supporté
    import llama_cpp
    print(f"   Version: {llama_cpp.__version__ if hasattr(llama_cpp, '__version__') else 'N/A'}")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 6: Gradio
print("\n6️⃣  Test Gradio (UI)...")
try:
    import gradio as gr
    print(f"✅ Gradio {gr.__version__} disponible")
except Exception as e:
    print(f"❌ Erreur: {e}")

print("\n" + "=" * 60)
print("✅ Tests de base terminés!")
print("=" * 60)
print("\n💡 Conseil: Vous pouvez maintenant utiliser les modules du projet.")
print("   - Construire un indice RAG: python -m src.rag_build")
print("   - Lancer l'UI: python ui/app.py")
print("   - Démo CLI: python demo_cli.py")

## 🎉 Installation terminée !

Votre environnement Agent Vocal IA est prêt. Vous pouvez maintenant :

1. **Construire des indices RAG** pour vos matières
2. **Tester les modules individuellement** (ASR, TTS, LLM)
3. **Lancer l'interface Gradio** pour une utilisation interactive
4. **Utiliser le mode CLI** pour des tests en ligne de commande

### Prochaines étapes

Consultez le `README.md` pour les instructions d'utilisation détaillées.